In [ ]:
pip install undetected_chromedriver

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd
import numpy as np
from selenium import webdriver
import undetected_chromedriver as uc
import gc
import csv

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
from datetime import datetime

# Ripley Web Scraping

En esta sección conseguiremos mediante web scraping de la página de Ripley. Nos centraremos en las categorías de vestimenta y calzado

In [ ]:
categorias=["hombre", "mujer", "moda-infantil","calzado"]

In [ ]:
# Funciones auxiliares
def acceso(url, browser):
    """
    Función para acceder a la página web y extraer el HTML
    """
    browser.get(url)
    browser.implicitly_wait(5)

    # Cierra las ventanas de suscripción y ubicación
    cerrar_ventanas(browser)

    html = browser.page_source
    soup = bs(html, "html.parser")
    return soup

def cerrar_ventanas(browser):
    """
    Función para cerrar las ventanas emergentes (suscripción, ubicación, etc.)
    """
    xpaths = [
        '//*[@class="align-right secondary slidedown-button"]',  # Ventana de suscripción
        '//*[@class="tooltip-button-close"]'  # Ventana de ubicación
    ]

    for xpath in xpaths:
        try:
            browser.find_element("xpath", xpath).click()
        except:
            pass  # Si no se encuentra el elemento, continúa con el siguiente xpath

def number_page(enlace, browser):
    """
    Función para obtener la cantidad de páginas de una categoría
    """
    soup = acceso(enlace, browser)
    articulos = int(soup.find("div", {"class": "col-xs-12 col-md-9 col-lg-9"}).text.split(" ")[0])
    return articulos // 48 + 1

def next_page(enlace, pagina):
    """
    Función para generar la URL de la página i
    """
    return f"{enlace}?page={pagina}"

def text_extract(textos):
    """
    Función para extraer el texto de los elementos encontrados
    """
    return [texto.text for texto in textos]

def prices_extract(precios):
    """
    Función para extraer el precio de los elementos encontrados
    """
    return [precio.text.strip()[3:] for precio in precios]

def limpiar_memoria():
    """
    Función para liberar memoria eliminando objetos innecesarios
    """
    gc.collect()

def obtener_datos_categoria(url, browser):
    """
    Generador para obtener datos de productos por categoría.
    Este generador ayudará a manejar grandes cantidades de datos de manera eficiente.
    """
    n = number_page(url, browser) + 1
    for i in range(1, n):
        url_page = next_page(url, i)
        time.sleep(random.uniform(2, 5))  # Pausa aleatoria entre 2 y 5 segundos
        soup = acceso(url_page, browser)

        items = soup.find_all("div", {"class": "catalog-product-details__name"})
        marcas = soup.find_all("div", {"class": "brand-logo"})
        precios = soup.find_all("li", {"title": "Precio Internet"})

        items_list = text_extract(items)
        marcas_list = text_extract(marcas)
        precios_list = prices_extract(precios)

        # Devuelve los resultados como un lote, y no los guarda en memoria
        yield items_list, marcas_list, precios_list

        # Liberar memoria después de procesar cada página
        del soup
        limpiar_memoria()

browser = uc.Chrome()

# Inicialización de listas globales para los datos de todas las categorías
items_totales = []
marcas_totales = []
precios_totales = []
categorias_totales = []

# Lista de categorías a procesar
categorias = ["hombre", "mujer", "moda-infantil", "calzado"]

# Escribir los datos en un archivo CSV de manera incremental
with open('Ripley.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Escribir encabezados en el archivo CSV
    writer.writerow(["Item", "Marca", "Precio", "Categoria", "Web", "Fecha"])

    # Recorrer cada categoría y obtener los datos
    for categoria in categorias:
        url = f"https://simple.ripley.com.pe/{categoria}"

        # Usar el generador para obtener los datos de cada página
        for items_list, marcas_list, precios_list in obtener_datos_categoria(url, browser):
            for item, marca, precio in zip(items_list, marcas_list, precios_list):
                # Escribir los datos de cada producto en el archivo CSV
                writer.writerow([item, marca, precio, categoria, "Ripley", datetime.now().strftime("%m/%d/%Y")])

            # Liberar memoria
            del items_list, marcas_list, precios_list
            gc.collect()

browser.quit()


